In [1]:
import pandas as pd
import findspark 
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Exemplo2").getOrCreate()
from pyspark.sql.functions import *
from pyspark.sql import functions as Func

Essa foi minha pesca feita localmente aqui no jupyter para depois levar pro AWS Glue.

Primeiro li o arquivo da trusted já em parquet, e coloquei em um dataframe spark.

In [2]:
df_spark = spark.read.format("parquet").load("completo.parquet")
#criando um dataframe spark com o arquivo parquet da trusted.

In [3]:
df_spark = df_spark.select('Id','Titulo','Popularidade','Lancamento','Nota','Votos','Orçamento','Receita','Duração')

# selecionando apenas os dados que pretendo usar.

In [4]:
df_spark = df_spark.select("*").where(Func.col("Lancamento").between("2008-01-01", "2023-12-31")).orderBy(Func.col("Lancamento"))

# Aqui foi feito um novo filtro nesses dados. organizando os filmes no periodo que foi escolhido. (As 4 fases marvel)

In [5]:
df_spark = df_spark.select("*").where(Func.col("Receita" and "Orçamento") != 0)

# Esse filtro tira os filmes da fase 4 que ainda não foram lançados ou que estão em cartaz. sem dados suficientes para analisar.

In [6]:
df_spark = df_spark.select("*").where(Func.col("Votos") > 2000)

# Mais um filtro que limita a quantidade de votos mínimos;

In [7]:
df_spark.show(100)

+------+--------------------+------------+----------+-----+-----+---------+----------+-------+
|    Id|              Titulo|Popularidade|Lancamento| Nota|Votos|Orçamento|   Receita|Duração|
+------+--------------------+------------+----------+-----+-----+---------+----------+-------+
|  1726|            Iron Man|      93.707|2008-04-30|7.639|25035|140000000| 585174222|    126|
|  1724| The Incredible Hulk|      65.123|2008-06-12|6.206|11093|150000000| 163712074|    114|
| 10138|          Iron Man 2|      99.198|2010-04-28|6.835|19878|200000000| 623933331|    124|
| 10195|                Thor|      82.222|2011-04-21|6.767|20132|150000000| 449326618|    115|
|  1771|Captain America: ...|      56.576|2011-07-22|6.995|20383|140000000| 370569774|    124|
| 24428|        The Avengers|     125.405|2012-04-25|7.711|29325|220000000|1518815515|    143|
| 68721|          Iron Man 3|      71.806|2013-04-18|6.927|21181|200000000|1215577205|    130|
| 76338|Thor: The Dark World|      75.254|2013-10-

In [8]:
#df_spark.createOrReplaceGlobalTempView("filmes")
#spark.sql("select * from global_temp.filmes").show()
#resultado = spark.sql("SELECT *, CASE WHEN Titulo LIKE 'Avengers%' THEN 'Não' ELSE 'Sim' END AS Solo FROM filmes")

#Aqui estava foi a tentativa de criar uma coluna que filtrasse os filmes solos dos herois.

In [9]:
df_spark = df_spark.withColumn("Vingadores", when(df_spark["Titulo"].contains("Avengers"), "Sim").otherwise("Não"))

#Filtro feito dessa maneira, onde criei uma coluna chamada "Vingadores" que me diz se esse filme é dos vingadores ou não.

In [16]:
df_spark.write.saveAsTable("df_sparktb")

In [15]:
#df_spark.write.format("parquet").save("C:\\Users\\victo\\Downloads\\DATA E ANALYTICS\\Documentos\\Sprint_9\\tabela\\")

In [17]:
spark.sql("show tables").show()

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|  default|  df_spark|      false|
|  default|df_sparktb|      false|
+---------+----------+-----------+



In [23]:
spark.sql("select * from df_sparktb").show()

+------+--------------------+------------+----------+-----+-----+---------+----------+-------+----------+
|    Id|              Titulo|Popularidade|Lancamento| Nota|Votos|Orçamento|   Receita|Duração|Vingadores|
+------+--------------------+------------+----------+-----+-----+---------+----------+-------+----------+
|  1726|            Iron Man|      93.707|2008-04-30|7.639|25035|140000000| 585174222|    126|       Não|
|  1724| The Incredible Hulk|      65.123|2008-06-12|6.206|11093|150000000| 163712074|    114|       Não|
| 10138|          Iron Man 2|      99.198|2010-04-28|6.835|19878|200000000| 623933331|    124|       Não|
| 10195|                Thor|      82.222|2011-04-21|6.767|20132|150000000| 449326618|    115|       Não|
|  1771|Captain America: ...|      56.576|2011-07-22|6.995|20383|140000000| 370569774|    124|       Não|
| 24428|        The Avengers|     125.405|2012-04-25|7.711|29325|220000000|1518815515|    143|       Sim|
| 68721|          Iron Man 3|      71.806|2013